In [16]:
def escalona_tempo(comex_df):
    
    mes_at=comex_df[(comex_df.CO_ANO==comex_df.CO_ANO.max())].CO_MES.max()
    
    mes=pd.DataFrame({'CO_MES':np.arange(1,13)})
    
    mes_i=pd.DataFrame({'mes_i':np.arange(1,13,1).astype(int)})
    
    bim =pd.DataFrame({'bim_i':np.arange(1,7,.5).astype(int)})
    
    trim=pd.DataFrame({'trim_i':np.arange(1,5,.34).astype(int)})
    
    quadri=pd.DataFrame({'quadri_i':np.arange(1,4,.25).astype(int)})
    
    sem=pd.DataFrame({'sem_i':np.arange(1,3,1/6).astype(int)})
    
    escala=pd.concat([
        mes,
        pd.concat([
            mes_i.shift(mes_at-12).iloc[:mes_at],
            mes_i.shift(mes_at).dropna()
        ]),
        pd.concat([
            bim.shift(mes_at-12).iloc[:mes_at],
            bim.shift(mes_at).dropna()
        ]),
            pd.concat([
            trim.shift(mes_at-12).iloc[:mes_at],
            trim.shift(mes_at).dropna()
        ]),
            pd.concat([
            quadri.shift(mes_at-12).iloc[:mes_at],
            quadri.shift(mes_at).dropna()
        ]),
            pd.concat([
            sem.shift(mes_at-12).iloc[:mes_at],
            sem.shift(mes_at).dropna()
        ])
    ],axis=1).astype(int)
    #print(escala)
    comex_df=comex_df.merge(escala)

    comex_df['ano_i']=comex_df.CO_ANO.shift(8)
    
    # Identifica a mês corrente
    mês_corrente=comex_df[comex_df.CO_ANO==comex_df.CO_ANO.max()].CO_MES.max()
    # Completa escalonamento criano ano índice
    comex_df['ano_i']=comex_df.CO_ANO.shift(mês_corrente)
    
    # Corrige na em ano_i
    k=comex_df[comex_df.ano_i.isna()]
    w=comex_df[~comex_df.ano_i.isna()]
    k=k.drop(columns='ano_i')
    z=np.array(w[w.index.isin(k.index+8)].ano_i-1)
    k['ano_i']=z
    comex_df=pd.concat([k,w])
    
    # Corrige tipo
    comex_df[comex_df.select_dtypes('float').columns]=comex_df[comex_df.select_dtypes('float').columns].astype(int)

    # Identifica escalas
    escalas=comex_df.columns[-6:-1]

    print(list(enumerate(escalas)))

    #si=int(input('Indice da escala:'))
    si=0

    global sc
    
    sc=escalas[si]
    
    comex_df=comex_df.groupby(by=['date','ano_i',sc])[quanti_interesse].sum().reset_index()#.astype(int)
    
    tab=comex_df[['date','ano_i',sc]]
    tab=tab.drop_duplicates(subset=['ano_i',sc])
    tab=tab[tab.ano_i!=tab.ano_i.min()]
    
    comex_df=tab.merge(comex_df)
    
    return comex_df

In [ ]:
print('** Função escalona_tempo(comex_df) disponível **')